<a href="https://colab.research.google.com/github/ptran1203/gan_project/blob/master/notebooks/BAGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create the link from this drive folder to your drive.

---


https://drive.google.com/drive/folders/1RNJXceXkNatuAbNn-CKB8MrgaEHG5RpM?usp=sharing

In [1]:
cd /content

/content


In [2]:
%tensorflow_version 1.x
from google.colab import drive, output
drive.mount('/content/drive')

!rm -rf '/content/gan_project'
!git clone https://github.com/ptran1203/gan_project
!pip install git+https://www.github.com/keras-team/keras-contrib.git
output.clear()

In [3]:
cd gan_project

/content/gan_project


In [4]:
BASE_DIR = '/content/drive/My Drive/bagan'
DS_DIR = '/content/drive/My Drive/bagan/dataset/chest_xray'
DS_SAVE_DIR = '/content/drive/My Drive/bagan/dataset/save'
gratio_mode = 'uniform'
dratio_mode = 'uniform'

from bagan import *
from batch_gen import *
from utils import *
from tensorflow.keras.layers import LayerNormalization

def create_dir_if_any(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

class Bagan(BalancingGAN):
    pass

    # def backup_point(self, epoch): return


class BatchGen(BatchGenerator):
    to_train_classes = INVERT_CATEGORIES_MAP
    to_test_classes = list(range(81, 86))


is_test = 0
## Test batch GEN
if is_test:
    bg = BatchGen(BatchGen.TRAIN, 64, 'multi_chest', 64)
    labels = np.array([0, 0, 1, 1, 2, 2, 3 ,1])
    samples = bg.ramdom_kshot_images(4, labels)
    print(samples.shape)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
gan_epochs  = 400000
adam_lr = 0.00002
batch_size = 64
# dataset_name should be "flowers", "chest", "multi_chest"
# "chest" is binary classification, "multi_chest" using chest-xray14 dataset
dataset_name = 'multi_chest'
latent_size = 128
# Use resnet architecture for Generator
resnet = False
# Use self-attention mechanism
attention = False
np.random.seed(0)
# Image resoulution
rst = 128
is_prune = False

prune = [900, 900] if is_prune else None

res_dir = BASE_DIR + '/result/bagan_{}_{}_v2'.format(dataset_name,rst)
create_dir_if_any(res_dir)

bg_train_full = BatchGen(BatchGen.TRAIN, batch_size, dataset_name, rst,prune_classes=prune)
bg_test = BatchGen(BatchGen.TEST, batch_size, dataset_name, rst)
channels = bg_train_full.dataset_x[0].shape[-1]
shape = bg_train_full.dataset_x[0].shape

print('img shape', shape)
classes = bg_train_full.get_label_table()
target_classes = np.array(range(len(classes)))
target_class_id = 0 # train to balance this class
print('Class counters: ', bg_train_full.per_class_count)
gan = Bagan(
    target_classes,
    target_class_id,
    adam_lr=adam_lr,
    latent_size=latent_size,
    res_dir=res_dir,
    image_shape=shape,
    min_latent_res=4,
    autoenc_epochs=50
)
gan.train(bg_train_full, bg_test, epochs=gan_epochs)


load data from /content/drive/My Drive/bagan/dataset/multi_chest/imgs_labels_128.pkl successfully
15830 15830
load data from /content/drive/My Drive/bagan/dataset/multi_chest/imgs_labels_128.pkl successfully
15830 15830
img shape (128, 128, 1)
Class counters:  [5000, 3341, 1475, 1384, 946, 767, 748, 458, 394, 382, 312, 254, 219, 112, 38]
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
BAGAN init_autoenc
BAGAN: training autoencoder
Autoencoder train epoch: 1/50

Autoencoder train epoch: 2/50


KeyboardInterrupt: ignored

In [ ]:
size = 1000 # images in each class
labels = [0] * size
for i in range(1, gan.nclasses):
    labels += [i] * size 

labels = np.array(labels)
fakes = gan.generate(labels)
show_samples(fakes[:10])
res = (fakes, labels)


In [ ]:
def evaluate_g(self, test_x, test_y):
    y_pre = self.combined.predict(test_x)
    y_pre = np.argmax(y_pre, axis=1)
    cm = metrics.confusion_matrix(y_true=test_y, y_pred=y_pre)  # shape=(12, 12)
    plt.figure()
    plot_confusion_matrix(cm, hide_ticks=True,cmap=plt.cm.Blues)
    plt.show()

def evaluate_d(self, test_x, test_y):
    y_pre = self.discriminator.predict(test_x)
    y_pre = np.argmax(y_pre, axis=1)
    cm = metrics.confusion_matrix(y_true=test_y, y_pred=y_pre)  # shape=(12, 12)
    plt.figure()
    plot_confusion_matrix(cm, hide_ticks=True,cmap=plt.cm.Blues)
    plt.show()

train_x, train_y = bg_train_full.dataset_x, bg_train_full.dataset_y
# train_y = train_y[:10]
# latent = gan.generate_latent(train_y)
# evaluate_g(gan, latent, train_y)
# evaluate_d(gan, train_x, train_y)

In [ ]:
# Generate image for data augmentation
def gen_for_class(self, bg, classid,size=1000):
    total = None
    for i in range(1000):
        labels = [classid] * size
        labels = np.array(labels)
        latent = self.generate_latent(labels)
        print("Predict...")
        gen = self.generator.predict(latent)
        d_outputs = self.discriminator.predict(gen)
        d_outputs = np.argmax(d_outputs, axis=1)
        print(Counter(d_outputs))
        to_keep = np.where(labels == d_outputs)[0]
        gen = gen[to_keep]
        if total is None:
            total = gen
        else:
            total = np.concatenate([total, gen], axis=0)
        
        if len(total) >= size:
            total = total[:size]
            break

    print("done class {}, size {}".format(classid, len(total)))
    return total, np.array([classid] * len(total))

def gen_augment_data(self, bg, size=1000):
    total = None
    labels = None
    for i in bg.classes:
        gen , label = gen_for_class(self,bg, i, size)
        if total is None:
            total = gen
            labels = label
        else:
            total = np.concatenate([total, gen], axis=0)
            labels = np.concatenate([labels, label], axis=0)
    
    print("Done all ", len(total))
    return total, labels
fakes = gen_augment_data(gan, bg_train_full,2000)


In [ ]:
show_samples(fakes[0][:10])
pickle_save(fakes, '/content/drive/My Drive/bagan/dataset/multi_chest/imgs_train_gen_v3.pkl')